# Chunking & Embeddings

Chunking and embedding are foundational steps in building a RAG system. Chunking involves dividing text into smaller, manageable parts for better retrieval and processing, while embedding involves converting these chunks into numerical vectors that represent their semantic meaning. These steps enable efficient information retrieval and precise answer generation.

### Chunking

Chunking involves splitting long documents into smaller, coherent segments of text, which can be retrieved and processed more effectively. This prevents large documents from being treated as a single retrieval unit, improving the system's ability to pinpoint relevant information.

#### Benefits of Chunking

- **Improved Retrieval Accuracy**: Smaller chunks make it easier to retrieve precise and relevant information for a query
- **Reduced Memory Overhead**: Dividing text into smaller segments ensures more efficient storage and processing
- **Facilitates Parallel Processing**: Each chunk can be processed independently, enhancing system efficiency
- **Enhanced Contextual Relevance**: Smaller chunks reduce noise, improving the likelihood of retrieving relevant content

#### Strategy
For our pipeline, we use the Recursive Character Text Splitter from LangChain, a robust method for splitting text based on character limits while preserving semantic integrity.

**Parameters**
- **Chunk Size**: The maximum number of characters in each chunk. A larger chunk size includes more context but risks including irrelevant information.
- **Chunk Overlap**: The number of overlapping characters between consecutive chunks. Overlap ensures continuity of context across chunks.

### Embeddings

Embeddings transform text into high-dimensional numerical vectors that encode semantic meaning. These embeddings are used to calculate similarity between queries and chunks, enabling effective retrieval of relevant contexts.

#### Models
We use two state-of-the-art embedding models in our experiments:

1. `text-embedding-3-large`

    - A large, general-purpose embedding model capable of capturing nuanced semantic meaning.

    - Suitable for complex queries requiring detailed contextual understanding.

2. `text-embedding-ada-002`

    - A lightweight, highly efficient model with competitive performance.
    
    - Ideal for scalable applications with a large number of documents or queries.

### Experiments

We experiment with different chunking strategies and embedding models to evaluate the best combinations. The strategies involve varying chunk sizes, overlaps, and embedding models:

| **Chunk Size** | **Overlap** | **Embedding Model**      | **Description**                                                                 |
|----------------|-------------|--------------------------|---------------------------------------------------------------------------------|
| 500            | 50          | `text-embedding-3-large` | Small chunk size with overlap to ensure continuity; tested with a large model. |
| 500            | 50          | `text-embedding-ada-002` | Same as above but using a more lightweight model for comparison.                    |
| 1000           | 0           | `text-embedding-ada-002` | Larger chunks with no overlap to assess performance on non-redundant segments. |
| 1000           | 100         | `text-embedding-3-large` | Larger chunks with overlap, paired with a nuanced embedding model.             |
| 2500           | 300         | `text-embedding-3-large` | Very large chunks with substantial overlap, aimed at preserving context.       |

The generated embeddings for each strategy are saved as pickle files for future retrieval and evaluation. These embeddings form the foundation for later stages of the RAG pipeline, including retrieval and answer generation.

## Imports

In [1]:
import pandas as pd
import pickle
import time
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import AzureOpenAI

import credentials

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import math

In [2]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-12-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

## Data Preparation and Chunking

Here we load the preprocessed data and chunk the texts with the defined parameters.

In [3]:
CHUNK_SIZE = 2500
CHUNK_OVERLAP = 300

In [4]:
def chunk_text(dataframe, text_column, chunk_size=1000, chunk_overlap=100):
    # Initialize RecursiveCharacterTextSplitter with dynamic parameters
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    # Split text in the specified column
    dataframe['content_chunks'] = dataframe[text_column].apply(lambda text: text_splitter.split_text(text))
    
    # Flatten the DataFrame for individual chunk rows
    chunked_df = dataframe.explode('content_chunks').reset_index(drop=True)

    return chunked_df

In [5]:
data = pd.read_csv('../data/processed/cleantech_processed.csv')
chunked_data = chunk_text(data, text_column='content', chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

print("Number of articles in the dataset: ", len(data))
print("Amount of chunks: ", len(chunked_data))

chunked_data.head()

Number of articles in the dataset:  9593
Amount of chunks:  21632


,title,date,content,domain,url,content_chunks
0,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Qatar Petroleum (QP) is targeting aggressive c...
1,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,Qatar Petroleum (QP) is targeting aggressive c...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"in the Phase 1 trains. Exxon Mobil, Royal Dutc..."
2,India Launches Its First 700 MW PHWR,2021-01-15,Nuclear Power Corp of India Ltd (NPCIL) synchr...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,Nuclear Power Corp of India Ltd (NPCIL) synchr...
3,India Launches Its First 700 MW PHWR,2021-01-15,Nuclear Power Corp of India Ltd (NPCIL) synchr...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,dropped out of the project and another two off...
4,New Chapter for US-China Energy Trade,2021-01-20,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,New US President Joe Biden took office this we...


Now that we have chunked the texts into smaller segments, the next step is to pass these chunks through an embedding model to obtain their vector representations. The embedding model maps the textual information into high-dimensional vector spaces, where semantic similarities and relationships are preserved.

## Embedding Creation

This code below generates embeddings for the text data in batches while handling rate-limiting errors. It splits the input data into manageable batches, each of size `BATCH_SIZE` and processes them concurrently using `ThreadPoolExecutor` with up to `MAX_WORKERS` threads. The `embed_batch` function sends each batch to the embedding model, implements retry logic to handle rate-limiting errors by waiting and retrying up to three times and collects the embeddings. The `embed_in_batches` function coordinates this process, maps batches to threads, and aggregates the results into a single list, ensuring each text chunk receives its embedding. The entire pipeline is designed for scalability and robustness against API limits.

In [ ]:
EMBEDDING_MODEL = "text-embedding-3-large"
BATCH_SIZE = 50
MAX_WORKERS = 8  # Number of threads
RETRY_DELAY = 60  # Retry delay in seconds for rate-limiting errors

# Function to generate embeddings for a batch of texts with retry logic
def embed_batch(text_batch, model, retries=3):
    attempts = 0
    while attempts < retries:
        try:
            response = client.embeddings.create(input=text_batch, model=model)
            return [item.embedding for item in response.data]
        except Exception as e:
            if "rate limit" in str(e).lower() or "429" in str(e):
                print(f"Rate limit exceeded. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)  # Wait before retrying
                attempts += 1
            else:
                print(f"Error embedding batch: {e}")
                return [None] * len(text_batch)
    print("Failed to embed after multiple attempts.")
    return [None] * len(text_batch)

# Function to process batches in parallel with rate limiting
def embed_in_batches(data, model=EMBEDDING_MODEL, batch_size=BATCH_SIZE, max_workers=MAX_WORKERS):
    total_chunks = len(data)
    num_batches = math.ceil(total_chunks / batch_size)
    
    # Create batches
    batches = [data[i * batch_size: (i + 1) * batch_size] for i in range(num_batches)]
    embeddings = [None] * total_chunks  # Placeholder for embeddings

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        for i, batch in enumerate(batches):
            futures[executor.submit(embed_batch, batch, model)] = i
        
        # Process results as they complete
        for future in tqdm(as_completed(futures), total=len(futures), desc="Embedding in Parallel"):
            batch_index = futures[future]
            try:
                batch_embeddings = future.result()
                embeddings[batch_index * batch_size: (batch_index + 1) * batch_size] = batch_embeddings
            except Exception as e:
                print(f"Error processing batch {batch_index}: {e}")
    
    return embeddings

# Prepare data for embedding generation
texts_to_embed = chunked_data["content_chunks"].tolist()

# Run batch embedding generation with rate limiting
chunked_data['embeddings'] = embed_in_batches(
    texts_to_embed, 
    model=EMBEDDING_MODEL,
    batch_size=BATCH_SIZE,
    max_workers=MAX_WORKERS
)

# Construct the filename dynamically
filename = f"../embeddings/recursive_{CHUNK_SIZE}_chunksize_{CHUNK_OVERLAP}_overlap_{EMBEDDING_MODEL.replace('-', '_')}.pkl"

# Save embeddings and corresponding chunked data to a file
with open(filename, 'wb') as f:
    pickle.dump(chunked_data, f)

print(f"Embeddings and chunked data saved to {filename}")

Embedding in Parallel:   1%|          | 4/433 [01:03<2:34:04, 21.55s/it]

Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...


Embedding in Parallel:   5%|▌         | 23/433 [05:06<1:08:56, 10.09s/it]

Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...


Embedding in Parallel:   6%|▌         | 25/433 [06:07<1:41:05, 14.87s/it]

...

Embedding in Parallel:  97%|█████████▋| 422/433 [2:03:25<02:14, 12.23s/it]

Rate limit exceeded. Retrying in 60 seconds...
Rate limit exceeded. Retrying in 60 seconds...


Embedding in Parallel:  98%|█████████▊| 424/433 [2:04:25<02:39, 17.69s/it]

Rate limit exceeded. Retrying in 60 seconds...


Embedding in Parallel: 100%|██████████| 433/433 [2:07:27<00:00, 17.66s/it]


The `chunked_data` DataFrame is structured so that each row represents an individual **chunk** of text derived from the original documents, with a corresponding **embedding vector** stored in the `embeddings` column. Each embedding vector is a dense list of floating-point numbers that encapsulates the semantic meaning of the text chunk.

These embeddings are critical for the RAG system, as they allow efficient similarity searches and retrieval tasks by representing the content of each text chunk in a high-dimensional vector space. When a query is embedded, the RAG system can quickly locate the most relevant text chunks by identifying embedding vectors in the database that are closest in meaning. This approach enables the system to retrieve contextually relevant information by comparing semantic relationships, streamlining the entire retrieval process.

We dynamically construct the filename based on the chunk size, overlap, and embedding model to ensure clarity and traceability in the saved file. Using `pickle`, we save the embeddings along with the corresponding chunked data to a file for later use. This approach allows us to efficiently store and retrieve preprocessed data, reducing the need for re-computation.